In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date
import dateutil.parser
from time import time
import io
import msoffcrypto
import math

In [2]:
passwd = 'pervasiveICU'

filename = '/home/jsenadesouza/DA-healthy2patient/Pervasive_Sensing_Enrollment_Log.xlsx'
decrypted_workbook = io.BytesIO()
with open(filename, 'rb') as file:
    office_file = msoffcrypto.OfficeFile(file)
    office_file.load_key(password=passwd)
    office_file.decrypt(decrypted_workbook)

ADAPT_enrollment = pd.read_excel(decrypted_workbook, engine="openpyxl")


In [3]:
data_input_file = "/home/jsenadesouza/DA-healthy2patient/results/outcomes/dataset/f10_t900_IntelligentICU_PAIN_ADAPT_15min.npz"
tmp = np.load(data_input_file, allow_pickle=True)
X= tmp['X']
y = tmp['y']
y_col_names = list(tmp['y_col_names'])
col_idx = y_col_names.index('patient_id')
y_patient = np.array(y[:, col_idx])
y_target = np.unique(y_patient)
print(len(y_target))

143


In [4]:
def set_patient_map():
    # create a map between the subject_deiden_id and the patient id
    patient_map = {}
    patient_enrollment = pd.read_excel('/data/daily_data/patient_id_mapping.xlsx', engine='openpyxl')

    for row in patient_enrollment.itertuples():
        patient_map[row.patient_id] = row.subject_deiden_id

    return patient_map


In [5]:
import glob
input_dir = '/data/datasets/ICU_Data/EHR_Data/truncated/2020-02-26/'
df_2016 = pd.read_csv(os.path.join(input_dir, 'encounters_0_trimmed.csv'))

df_2021 = []
files_enc = glob.glob('/data/daily_data/*/encounters*.csv',
                      recursive=True)
files_peso = glob.glob('/data/daily_data/*/height_weight*.csv',
                      recursive=True)

for file in files_enc:
    df = pd.read_csv(file)
    df_2021.append(df)
    
df_2021_peso = []
for file in files_peso:
    df = pd.read_csv(file)
    df_2021_peso.append(df)


df_2021 = pd.concat(df_2021)
df_2021_peso = pd.concat(df_2021_peso)

/tmp/ipykernel_5659/3250153617.py:12: DtypeWarning: Columns (15,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/tmp/ipykernel_5659/3250153617.py:12: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/tmp/ipykernel_5659/3250153617.py:12: DtypeWarning: Columns (15,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/tmp/ipykernel_5659/3250153617.py:12: DtypeWarning: Columns (38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/tmp/ipykernel_5659/3250153617.py:12: DtypeWarning: Columns (11,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/tmp/ipykernel_5659/3250153617.py:12: DtypeWarning: Columns (38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


In [6]:
patient_map = set_patient_map()
#df_2021_peso.columns
#row_peso = df_2021_peso[(df_2021_peso['patient_deiden_id'] == patient_map["P003"]) & (df_2021_peso['measurement_name'] == 'height_cm')]['measurement_value'].values[0]

In [7]:
df_2021.columns

Index(['hospital', 'encounter_effective_date', 'admit_datetime',
       'admit_source', 'dischg_datetime', 'sex', 'race', 'ethnicity',
       'language', 'city', 'country', 'zip', 'state', 'county',
       'marital_status', 'ed_arrival_datetime', 'isdead_y_n', 'encounter_type',
       'appointment_status', 'patient_type', 'hosp_svc_dept', 'hosp_svc_desc',
       'location_of_svc', 'external_name', 'birth_date', 'payer',
       'icd9_admit_diagnosis', 'icd9_admit_diagnosis_desc',
       'icd9_admit_diagnosis_group', 'icd10_admit_diagnosis',
       'icd10_admit_diagnosis_desc', 'icd10_admit_diagnosis_group',
       'admit_priority', 'seen_in_ed_y_n', 'dischg_disposition',
       'dischg_svc_dept_ip_obs', 'dischg_svc_desc_ip_obs', 'death_indicator',
       'death_date', 'aids', 'cancer', 'cerebrovascular_disease', 'chf',
       'copd', 'dementia', 'diabetes_w_o_complications',
       'diabetes_w_complications', 'm_i', 'metastatic_carcinoma',
       'mild_liver_disease', 'moderate_severe_l

In [8]:
patients_char = []
patient_map = set_patient_map()
for patient_id in y_target:
    if "P" in patient_id or "I" in patient_id:
        row = df_2021[df_2021['patient_deiden_id'] == patient_map[patient_id]]
        height = np.mean(df_2021_peso[(df_2021_peso['patient_deiden_id'] == patient_map[patient_id]) & (df_2021_peso['measurement_name'] == 'weight_kgs')]['measurement_value'].values)
        weight = df_2021_peso[(df_2021_peso['patient_deiden_id'] == patient_map[patient_id]) & (df_2021_peso['measurement_name'] == 'height_cm')]['measurement_value'].values[0]
        if "I" in patient_id:
            try:
                admit = datetime.strptime(ADAPT_enrollment['ICU_admit'][ADAPT_enrollment["Record ID"] == patient_id].values[0], '%m/%d/%y %H%M')
            except:
                try:
                    admit = datetime.strptime(ADAPT_enrollment['ICU_admit'][ADAPT_enrollment["Record ID"] == patient_id].values[0], '%m/%d/%Y %H%M')
                except:
                    print(f"admit: {ADAPT_enrollment['ICU_admit'][ADAPT_enrollment['Record ID'] == patient_id].values}")
            try:
                consent = pd.Timestamp(ADAPT_enrollment['Consent Date'][ADAPT_enrollment["Record ID"] == patient_id].values[0])
            except:
                print(f"consent: {ADAPT_enrollment['Consent Date'][ADAPT_enrollment['Record ID'] == patient_id].values}")
            try:
                dischg = datetime.strptime(ADAPT_enrollment['ICU_dischg'][ADAPT_enrollment["Record ID"] == patient_id].values[0], '%m/%d/%y %H%M')
            except:
                d_time = ADAPT_enrollment['ICU_dischg'][ADAPT_enrollment["Record ID"] == patient_id].values[0]
                if type(d_time) == type(datetime):
                    dischg = d_time
                else:              
                    dischg = datetime.combine(date.today(), datetime.min.time())
        else:
            ad_rows = row['admit_datetime'][~row['admit_datetime'].isna()].values
            dc_rows = row['dischg_datetime'][~row['dischg_datetime'].isna()].values
            try:
                admit = datetime.strptime(min(ad_rows), '%Y-%m-%d %H:%M:%S')
            except:
                admit = datetime.strptime(min(ad_rows), '%Y-%m-%d')
            try:
                dischg = datetime.strptime(max(dc_rows), '%Y-%m-%d %H:%M:%S')
            except:
                try:
                    dischg = datetime.strptime(max(dc_rows), '%Y-%m-%d')
                except:
                    print(patient_id)
                    print(dc_rows)
                    dischg = datetime.combine(date.today(), datetime.min.time())
        consent = admit
    else:
        row = df_2016[df_2016['record_id'] == int(patient_id)]
        height = row['height_cm'][~row['height_cm'].isna()].values[0]
        weight = row['weight_kgs'][~row['weight_kgs'].isna()].values[0]
        ad_rows = row['admit_datetime'][~row['admit_datetime'].isna()].values
        dc_rows = row['dischg_datetime'][~row['dischg_datetime'].isna()].values
        try:
            admit = datetime.strptime(min(ad_rows), '%Y-%m-%d %H:%M:%S')
        except:
            admit = datetime.strptime(min(ad_rows), '%Y-%m-%d')
        try:
            dischg = datetime.strptime(max(dc_rows), '%Y-%m-%d %H:%M:%S')
        except:
            try:
                dischg = datetime.strptime(max(dc_rows), '%Y-%m-%d')
            except:
                print(patient_id)
                print(dc_rows)
                dischg = datetime.combine(date.today(), datetime.min.time())
        consent = admit
        
        
    birth = datetime.strptime(row['birth_date'][~row['birth_date'].isna()].values[0], '%Y-%m-%d')
    age = int((consent - birth).days/365)
    lenght_stay = abs((dischg - admit).days)

    gender = row['sex'][~row['sex'].isna()].values[0]
    race = row['race'][~row['race'].isna()].values[0]
    ethnicity = row['ethnicity'][~row['ethnicity'].isna()].values[0]
    if len(row['aids'][~row['aids'].isna()]) > 0:
        aids = row['aids'][~row['aids'].isna()].values[0]
    else:
        aids = -1
    if len(row['cancer'][~row['cancer'].isna()]) > 0:
        cancer = row['cancer'][~row['cancer'].isna()].values[0]
    else:
        cancer = -1
    if len(row['cerebrovascular_disease'][~row['cerebrovascular_disease'].isna()]) > 0:
        cerebrovascular_disease = row['cerebrovascular_disease'][~row['cerebrovascular_disease'].isna()].values[0]
    else:
        cerebrovascular_disease = -1
    if len(row['dementia'][~row['dementia'].isna()]) > 0:
        dementia = row['dementia'][~row['dementia'].isna()].values[0]
    else:
        dementia = -1
    if len(row['paraplegia_hemiplegia'][~row['paraplegia_hemiplegia'].isna()]) > 0:
        paraplegia_hemiplegia = row['paraplegia_hemiplegia'][~row['paraplegia_hemiplegia'].isna()].values[0]
    else:
        paraplegia_hemiplegia = -1
    if len(row['smoking_status'][~row['smoking_status'].isna()]) > 0:
        smoking_status = row['smoking_status'][~row['smoking_status'].isna()].values[0]
    else:
        smoking_status = -1
    if len(row['chf'][~row['chf'].isna()]) > 0:
        chf = row['chf'][~row['chf'].isna()].values[0]
    else:
        chf = -1
    if len(row['copd'][~row['copd'].isna()]) > 0:
        copd = row['copd'][~row['copd'].isna()].values[0]
    else:
        copd = -1
    
    if len(row['diabetes_w_o_complications'][~row['diabetes_w_o_complications'].isna()]) > 0:
        diabetes_w_o_complications = row['diabetes_w_o_complications'][~row['diabetes_w_o_complications'].isna()].values[0]
    else:
        diabetes_w_o_complications = -1    
    if len(row['diabetes_w_complications'][~row['diabetes_w_complications'].isna()]) > 0:
        diabetes_w_complications = row['diabetes_w_complications'][~row['diabetes_w_complications'].isna()].values[0]
    else:
        diabetes_w_complications = -1    
    if diabetes_w_o_complications == 1 or diabetes_w_complications == 1:
        diabetes = 1
    elif diabetes_w_o_complications == -1 and diabetes_w_complications == -1:
        diabetes = -1
    elif diabetes_w_o_complications == 0 and diabetes_w_complications == 0:
        diabetes = 0
    if len(row['m_i'][~row['m_i'].isna()]) > 0:
        m_i = row['m_i'][~row['m_i'].isna()].values[0]
    else:
        m_i = -1
    if len(row['metastatic_carcinoma'][~row['metastatic_carcinoma'].isna()]) > 0:
        metastatic_carcinoma = row['metastatic_carcinoma'][~row['metastatic_carcinoma'].isna()].values[0]
    else:
        metastatic_carcinoma = -1
    if len(row['mild_liver_disease'][~row['mild_liver_disease'].isna()]) > 0:
        mild_liver_disease = row['mild_liver_disease'][~row['mild_liver_disease'].isna()].values[0]
    else:
        mild_liver_disease = -1
    if len(row['moderate_severe_liver_disease'][~row['moderate_severe_liver_disease'].isna()]) > 0:
        moderate_severe_liver_disease = row['moderate_severe_liver_disease'][~row['moderate_severe_liver_disease'].isna()].values[0]
    else:
        moderate_severe_liver_disease = -1
    if mild_liver_disease == 1 or moderate_severe_liver_disease == 1:
        liver_disease = 1
    elif mild_liver_disease == -1 and moderate_severe_liver_disease == -1:
        liver_disease = -1
    elif mild_liver_disease == 0 and moderate_severe_liver_disease == 0:
        liver_disease = 0
    if len(row['peptic_ulcer_disease'][~row['peptic_ulcer_disease'].isna()]) > 0:
        peptic_ulcer_disease = row['peptic_ulcer_disease'][~row['peptic_ulcer_disease'].isna()].values[0]
    else:
        peptic_ulcer_disease = -1
    if len(row['peripheral_vascular_disease'][~row['peripheral_vascular_disease'].isna()]) > 0:
        peripheral_vascular_disease = row['peripheral_vascular_disease'][~row['peripheral_vascular_disease'].isna()].values[0]
    else:
        peripheral_vascular_disease = -1
    if len(row['renal_disease'][~row['renal_disease'].isna()]) > 0:
        renal_disease = row['renal_disease'][~row['renal_disease'].isna()].values[0]
    else:
        renal_disease = -1
    if len(row['rheumatologic_disease'][~row['rheumatologic_disease'].isna()]) > 0:
        rheumatologic_disease = row['rheumatologic_disease'][~row['rheumatologic_disease'].isna()].values[0]
    else:
        rheumatologic_disease = -1


    patients_char.append({'patient_id': patient_id, 'sex': gender, 'race': race, 'height_cm': height, 
                          'age':age, 'weight_kgs':weight, 'lenght_stay':lenght_stay, 
                          "ethnicity":ethnicity, "aids":aids, "cancer":cancer, "cerebrovascular_disease":cerebrovascular_disease,
                          "dementia":dementia, "paraplegia_hemiplegia":paraplegia_hemiplegia, "smoking_status":smoking_status,
                          "chf":chf, "copd":copd, "diabetes":diabetes, "m_i":m_i, "metastatic_carcinoma":metastatic_carcinoma, 
                          "liver_disease":liver_disease, "peptic_ulcer_disease":peptic_ulcer_disease, "renal_disease":renal_disease, 
                          "rheumatologic_disease":rheumatologic_disease
                         })



df_char = pd.DataFrame(data=patients_char)

P070
[]


In [9]:
df_char.filter(["lenght_stay","patient_id"]).to_csv("los.csv")

In [10]:
print("age")
print(df_char["age"].describe())
print("height_cm")
print(df_char["height_cm"].describe())
print("weight_kgs")
print(df_char["weight_kgs"].describe())
print("lenght_stay")
print(df_char["lenght_stay"].describe())
for col in df_char.columns:
    print(col)
    print(df_char[col].value_counts())

age
count    143.000000
mean      58.377622
std       16.273977
min       18.000000
25%       49.000000
50%       61.000000
75%       71.000000
max       94.000000
Name: age, dtype: float64
height_cm
count    143.000000
mean     133.822936
std       46.373132
min       50.008518
25%       84.298874
50%      157.479996
75%      175.139901
max      195.580002
Name: height_cm, dtype: float64
weight_kgs
count    143.000000
mean     125.946022
std       48.081075
min       47.627160
25%       81.646560
50%      120.003433
75%      173.989998
max      193.039993
Name: weight_kgs, dtype: float64
lenght_stay
count    143.000000
mean      57.685315
std       88.966794
min        2.000000
25%       11.000000
50%       20.000000
75%       57.500000
max      411.000000
Name: lenght_stay, dtype: float64
patient_id
100      1
I037A    1
I027A    1
I028A    1
I031A    1
        ..
59       1
60       1
61       1
63       1
P070     1
Name: patient_id, Length: 143, dtype: int64
sex
MALE      94
FEMAL

In [11]:
df_char.loc[df_char.sex == 'MALE', 'sex']= 0
df_char.loc[df_char.sex != 'MALE', 'sex']= 1
df_char.loc[df_char.race == 'BLACK', 'race']= 0
df_char.loc[df_char.race != 'BLACK', 'race']= 1

In [12]:
len(np.unique(df_char["patient_id"].values))

143

In [13]:
X_char = []
col_patient = y_col_names.index('patient_id')
for xx, sample in zip(X.squeeze(), y):
    try:
        char_pat = df_char[df_char["patient_id"] == sample[col_patient]]
        char_final = list(char_pat.loc[:, char_pat.columns != "patient_id"].values[0])
        xx = np.transpose(xx, (1,0))
        poi = POI(xx)

        char_final.append(poi)
        X_char.append(char_final)
    except:
        print(sample[col_patient])
X_char = np.array(X_char)

44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
22
22
22
22
22
22
22
22
22
22
22
106
106
106
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
55
55
55
55
55
55
55
55
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
36
36
36
36
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
59
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
35
35
35
35
35
35
35
35
35
35
35
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
91
9

80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
98
98
98
98
98
67
67
67
67
67
67
67
67
67
67
67
67
63
63
63
63
63
63
63
63
63
63
63
63
63
63
63
63
63
63
63
63
63
63
63
48
48
48
48
48
48
48
48
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
112
112
112
112
112
112
112
112
112
112
112
112
112
90
90
90
90
90
90
90
90
90
90
90
90
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
93
93
93
93
93
93
93
93
93

P054
P054
P054
P054
P054
P054
P054
P054
P054
P054
P003
P003
P003
P003
P003
P003
P003
P003
P003
P003
P003
P003
P009
P009
P006
P006
P006
P006
P070
P070
P070
P070
P070
P039
P042
P042
P042
P042
P042
P042
P042
P042
P042
P042
P042
P042
P042
P037
P037
P037
P037
P037
P037
P037
P037
P037
P037
P037
P037
P037
P037
P037
P037
P037
P037
P037
P037
P037
P037
P037
P007
P007
P007
P007
P024
P024
P024
P024
P024
P024
P024
P024
P004
P004
P004
P004
P004
P004
P004
P004
P004
P004
P004
P004
P004
P004
P004
P004
P004
P004
P004
P004
P004
P004
P004
P004
P004
P004
P004
P004
P004
P004
P004
P004
P004
P004
P004
P004
P004
P004
P004
P004
P004
P004
P004
P004
P028
P028
P028
P028
P028
P028
P028
P028
P028
P028
P028
P028
P028
P028
P028
P028
P028
P028
P028
P028
P028
P028
P063
P063
P063
P063
P021
P021
P021
P021
P021
P021
P017
P017
P017
P017
P017
P017
P046
P046
P046
I036A
I036A
I036A
I034A
I034A
I034A
I034A
I034A
I034A
I034A
I034A
I034A
I034A
I034A
I034A
I034A
I034A
I034A
I034A
I034A
I034A
I034A
I034A
I034A
I034A
I034A
I034A
I03

In [14]:
print(y_col_names)

['heart_rate', 'heart_rate_class', 'temp', 'temp_class', 'lenght_of_stay', 'is_dead', 'pain_score', 'pain_score_class', 'sofa_score', 'sofa_score_class', 'map', 'map_class', 'braden_score', 'braden_score_class', 'spo2', 'spo2_class', 'cam', 'patient_id']


In [15]:
col_idx = y_col_names.index('pain_score')
pain_scores = np.array(y[:, col_idx])
pain_scores = [round(float(x)) for x in pain_scores]
print(np.unique(pain_scores, return_counts=True))

(array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]), array([ 265, 4929,   68,   89,   66,   69,   91,   78,  158,  105,   43,
         46]))


In [16]:
print(4959+   68+   89+   66+   69)

5251


In [17]:
print(91+ 78 + 158+ 105+ 43 +46)

521
